# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admission_predict`
* table: `admission_predict`

In [53]:
# your code here

admisions_predict = pd.read_csv("data/Admission_Predict.csv")

Remove trailing spaces at the end of the column names if there are any.


In [54]:
# your code here
admisions_predict.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [55]:
headers_update = ['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit']

In [56]:
headers_update = [h.strip() for h in headers_update]

In [57]:
headers_update

['Serial No.',
 'GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR',
 'CGPA',
 'Research',
 'Chance of Admit']

In [58]:
admisions_predict.columns = headers_update

Let's evaluate the dataset by looking at the `head` function.

In [59]:
# your code here

admisions_predict.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [9]:
# your code here

admisions_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         385 non-null    int64  
 1   GRE Score          385 non-null    int64  
 2   TOEFL Score        385 non-null    int64  
 3   University Rating  385 non-null    int64  
 4   SOP                385 non-null    float64
 5   LOR                385 non-null    float64
 6   CGPA               385 non-null    float64
 7   Research           385 non-null    int64  
 8   Chance of Admit    385 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 27.2 KB


Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [10]:
# your code here

admisions_predict = admisions_predict.set_index('Serial No.')
#admisions_predict

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [11]:
# your code here

# they are not unqiue
admisions_predict["GRE Score"].duplicated()

Serial No.
1      False
2      False
3      False
4      False
5      False
       ...  
381     True
382     True
383     True
384     True
385     True
Name: GRE Score, Length: 385, dtype: bool

In [12]:
admisions_predict["CGPA"].duplicated()

Serial No.
1      False
2      False
3      False
4      False
5      False
       ...  
381     True
382     True
383     True
384     True
385     True
Name: CGPA, Length: 385, dtype: bool

In [13]:
admisions_predict["GRE Score"].value_counts()

324    16
312    16
314    14
322    14
321    13
316    12
327    12
317    12
320    12
311    11
325    11
313    11
308    11
323    10
315    10
326    10
329     9
319     9
318     9
300     9
310     8
331     8
307     8
304     8
301     8
299     8
305     7
298     7
340     7
334     7
330     6
332     6
306     6
328     6
296     5
309     5
303     5
302     5
336     5
335     4
333     4
295     4
338     4
297     4
339     3
294     2
290     2
337     1
293     1
Name: GRE Score, dtype: int64

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [14]:
# your code here

admisions_predict["GRE Score"]=admisions_predict["GRE Score"].astype(str) 

In [15]:
admisions_predict["CGPA"]=admisions_predict["CGPA"].astype(str) 

In [16]:
admisions_predict["GRE-CGPA_scores"] = admisions_predict["GRE Score"] + "-" + admisions_predict["CGPA"]

In [17]:
admisions_predict.drop(["GRE Score", "CGPA"], axis = 1,  inplace = True)

In [18]:
admisions_predict.reset_index()

,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE-CGPA_scores
0,1,118,4,4.5,4.5,1,0.92,337-9.65
1,2,104,3,3.0,3.5,1,0.72,316-8.0
2,3,110,3,3.5,2.5,1,0.80,322-8.67
3,4,103,2,2.0,3.0,0,0.65,314-8.21
4,5,115,5,4.5,3.0,1,0.90,330-9.34
...,...,...,...,...,...,...,...,...
380,381,110,3,3.5,3.5,1,0.82,324-9.04
381,382,107,3,3.0,3.5,1,0.84,325-9.11
382,383,116,4,5.0,4.5,1,0.91,330-9.45
383,384,103,3,3.5,4.0,0,0.67,312-8.78


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [19]:
# your code here
CGPA = admisions_predict["GRE-CGPA_scores"].str.split("-").str[1]

In [20]:
CGPA_col = pd.Series(CGPA).astype(float)

In [21]:
admisions_predict["CGPA_score"] = CGPA_col

In [22]:
admisions_predict

,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE-CGPA_scores,CGPA_score
Serial No.,,,,,,,,
1,118,4,4.5,4.5,1,0.92,337-9.65,9.65
2,104,3,3.0,3.5,1,0.72,316-8.0,8.00
3,110,3,3.5,2.5,1,0.80,322-8.67,8.67
4,103,2,2.0,3.0,0,0.65,314-8.21,8.21
5,115,5,4.5,3.0,1,0.90,330-9.34,9.34
...,...,...,...,...,...,...,...,...
381,110,3,3.5,3.5,1,0.82,324-9.04,9.04
382,107,3,3.0,3.5,1,0.84,325-9.11,9.11
383,116,4,5.0,4.5,1,0.91,330-9.45,9.45


In [23]:
admisions_predict[admisions_predict["CGPA_score"] > 9.00]

,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE-CGPA_scores,CGPA_score
Serial No.,,,,,,,,
1,118,4,4.5,4.5,1,0.92,337-9.65,9.65
5,115,5,4.5,3.0,1,0.90,330-9.34,9.34
11,112,4,4.0,4.5,1,0.78,328-9.1,9.10
20,116,5,5.0,5.0,1,0.94,328-9.5,9.50
21,119,5,5.0,4.5,1,0.95,334-9.7,9.70
...,...,...,...,...,...,...,...,...
380,111,4,4.5,4.0,1,0.89,329-9.23,9.23
381,110,3,3.5,3.5,1,0.82,324-9.04,9.04
382,107,3,3.0,3.5,1,0.84,325-9.11,9.11


In [24]:
admisions_predict[(admisions_predict["CGPA_score"] > 9.00) & (admisions_predict["Research"] == 1)]

,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,GRE-CGPA_scores,CGPA_score
Serial No.,,,,,,,,
1,118,4,4.5,4.5,1,0.92,337-9.65,9.65
5,115,5,4.5,3.0,1,0.90,330-9.34,9.34
11,112,4,4.0,4.5,1,0.78,328-9.1,9.10
20,116,5,5.0,5.0,1,0.94,328-9.5,9.50
21,119,5,5.0,4.5,1,0.95,334-9.7,9.70
...,...,...,...,...,...,...,...,...
380,111,4,4.5,4.0,1,0.89,329-9.23,9.23
381,110,3,3.5,3.5,1,0.82,324-9.04,9.04
382,107,3,3.0,3.5,1,0.84,325-9.11,9.11


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [25]:
# your code here

CGPA_SOAP = admisions_predict[(admisions_predict["CGPA_score"] > 9.00) & (admisions_predict["SOP"] < 3.5)]

In [26]:
CGPA_SOAP["Chance of Admit"].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [27]:
def standardize(col):
    
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """  
    
    # your code here
    
    std_column = admisions_predict[col]-admisions_predict[col].mean()/admisions_predict[col].std()
    return std_column

In [60]:
CGPA_std = standardize("CGPA")
GRE_std = standardize("GRE Score")
LOR_std = standardize("LOR")

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [66]:
# your code here
admisions_predict["CGPA_std"] = CGPA_std
admisions_predict["GRE_std"] = GRE_std
admisions_predict["LOR_std"] = LOR_std

We will generate the decision choice at random using the code below. Please run the cell.

In [63]:
# Libraries
from random import choices

In [67]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [69]:
# your code here

admisions_predict["deciding_column"] = admissions.lookup(admissions.index, decision_choice)

In [70]:
admisions_predict

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,1,337,118,4,4.5,4.5,9.65,1,0.92,-4.621747,309.598899,0.674568,0.674568
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-6.271747,288.598899,-0.325432,-6.271747
2,3,322,110,3,3.5,2.5,8.67,1,0.80,-5.601747,294.598899,-1.325432,294.598899
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-6.061747,286.598899,-0.825432,-0.825432
4,5,330,115,5,4.5,3.0,9.34,1,0.90,-4.931747,302.598899,-0.825432,-4.931747
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,-5.231747,296.598899,-0.325432,-5.231747
381,382,325,107,3,3.0,3.5,9.11,1,0.84,-5.161747,297.598899,-0.325432,297.598899
382,383,330,116,4,5.0,4.5,9.45,1,0.91,-4.821747,302.598899,0.674568,302.598899
383,384,312,103,3,3.5,4.0,8.78,0,0.67,-5.491747,284.598899,0.174568,0.174568


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [71]:
# your code here
admisions_predict["decision"] = np.where(admisions_predict["deciding_column"] > 0.8, 1, 0)

In [72]:
admisions_predict

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,-4.621747,309.598899,0.674568,0.674568,0
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-6.271747,288.598899,-0.325432,-6.271747,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,-5.601747,294.598899,-1.325432,294.598899,1
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-6.061747,286.598899,-0.825432,-0.825432,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,-4.931747,302.598899,-0.825432,-4.931747,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,-5.231747,296.598899,-0.325432,-5.231747,0
381,382,325,107,3,3.0,3.5,9.11,1,0.84,-5.161747,297.598899,-0.325432,297.598899,1
382,383,330,116,4,5.0,4.5,9.45,1,0.91,-4.821747,302.598899,0.674568,302.598899,1
383,384,312,103,3,3.5,4.0,8.78,0,0.67,-5.491747,284.598899,0.174568,0.174568,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [73]:
# your code here

admisions_predict["decision"].value_counts()

0    243
1    142
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [ ]:
# your code here

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [ ]:
# your code here